# ANALISIS FORECAST - Sesión 06

- Expositor: César Manuel Quezada Balcazar
- Email: cesar.quezada@dmc.pe
- LinkedIn: www.linkedin.com/in/quezada
- Tema: Modelos de Pronostico - Series de Tiempo

**Importamos los módulos necesarios**

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import requests
import io
import os
import pylab 
from dateutil.parser import parse
import scipy.stats as stats
from scipy.stats import kstest 
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

**Leemos nuestra serie de tiempo desde un archvo csv**

In [ ]:
df_agricola = pd.read_csv('datos/agricola.csv', sep = ';')
df_agricola.head(10)

In [ ]:
# Reaccinamos el index
df_agricola = df_agricola.set_index('fecha')

In [ ]:
plt.figure(figsize=(16,5), dpi=100)
plt.plot(df_agricola.papa, color = 'blue')
plt.show()

**(1) Particion la serie training/testing**

In [ ]:
tss = TimeSeriesSplit(n_splits = 12)

In [ ]:
X = df_agricola.drop(labels=['papa'], axis=1)
y = df_agricola['papa']

In [ ]:
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
# Graficamos los periodos train y test
plt.figure(figsize=(16,5), dpi=100)
plt.plot(pd.DataFrame(y_train).index, pd.DataFrame(y_train).papa,color='tab:blue')
plt.plot(pd.DataFrame(y_test).index, pd.DataFrame(y_test).papa,color='tab:red')
plt.xlabel('Periodos')
plt.ylabel('Precios de papa')
plt.show()

**(1) Proceso AUTO ARIMA**

In [ ]:
import pmdarima as pm

In [ ]:
model_auto = pm.auto_arima(y_train,
                           seasonal=True,
                           m = 12,
                           D=1)

In [ ]:
results = model_auto.fit(y_train) #ajustamos el modelo

In [ ]:
forecast_autoarima = model_auto.predict(n_periods=12)
forecast_autoarima

In [ ]:
forecast_autoarima = pd.DataFrame(forecast_autoarima)
forecast_autoarima.index = y_test.index
forecast_autoarima.columns = ['forecast']
forecast_autoarima.head(4)

In [ ]:
# plot
plt.figure(figsize=(16,5), dpi=100)
forecast_autoarima.plot(color = 'green', legend = True)
y_test.plot(color='red',legend = True)
plt.show()

Prediciendo valores futuros

In [ ]:
# model sarimax
model_sarimax = SARIMAX(df_agricola.papa, order= (1,0,0), seasonal_order=(0,1,0,12),
                       enforce_stationarity=False,
                      enforce_invertibility=False)

In [ ]:
results_sarimax = model_sarimax.fit() # ajuste del modelo

In [ ]:
forecast_sarimax = results_sarimax.get_forecast(steps = 12).predicted_mean.rename('forecast sarimax') # predicción horizonte futuro

In [ ]:
forecast_sarimax

In [ ]:
# Re estructuramos para poder graficar
history = df_agricola.papa
history = history.reset_index()
history

In [ ]:
# plot multiple
plt.figure(figsize=(16,5), dpi=100)
history.papa.plot(color = 'blue', legend = True)
forecast_sarimax.plot(color = 'green', legend = True)
plt.show()

**(2) Proceso SARIMAX**

In [ ]:
x1 = pd.DataFrame(y_train).shift(1)
x2 = pd.DataFrame(y_train).shift(2)
x1.columns = ['x1']
x2.columns = ['x2']

In [ ]:
# concatenando en regressors
regressors = pd.concat([x1, x2], axis=1)
regressors.head(5)

In [ ]:
regressors['x1'] = regressors.x1.fillna(190.59)
regressors['x2'] = regressors.x2.fillna(190.59)

In [ ]:
# corremos un nuevo autoarima con regresores
model_auto2 = pm.auto_arima(y_train,
                            seasonal=True,
                            d=1,
                            D=1,
                            start_P = 1,
                            start_Q = 1,
                            m = 12,
                            information_criterion = 'aic',
                            suppress_warnings=True,
                            stepwise=True,
                            exogenous = regressors.values)

In [ ]:
results2 = model_auto2.fit(y_train) #ajustamos el modelo

In [ ]:
forecast_autoarima = model_auto2.predict(n_periods=12)
forecast_autoarima = pd.DataFrame(forecast_autoarima)
forecast_autoarima.index = y_test.index
forecast_autoarima.columns = ['forecast']
forecast_autoarima.head(4)

In [ ]:
# plot
plt.figure(figsize=(16,5), dpi=100)
forecast_autoarima.plot(color = 'green', legend = True)
y_test.plot(color='red',legend = True)
plt.show()